In [ ]:
words = open("names.txt", 'r').read().splitlines()

In [ ]:
b = {}
for w in words:
    # Add start and ending characters
    chs = ["<S>"] + list(w) + ["<E>"] 
    # Get pairs of characters in a sliding window and add to dict
    for ch1, ch2 in zip(chs, chs[1:]):
        bigram = (ch1, ch2)
        b[bigram] = b.get(bigram, 0) + 1

In [ ]:
# Sort by highest occurence
sorted(b.items(), key = lambda kv: kv[1], reverse = True)

In [ ]:
import torch

In [ ]:
N = torch.zeros((27, 27), dtype = torch.int32)

In [ ]:
# Get all letters of the alphabet in a list
chars = sorted(list(set(''.join(words))))

# Create mapping
s_to_i = {s:i for i, s in enumerate(chars, start=1)}
s_to_i['.'] = 0
s_to_i

# Reverse Mapping
i_to_s = {i:s for s, i in s_to_i.items()}
i_to_s

In [ ]:
for w in words:
    # Add start and ending characters
    chs = ["."] + list(w) + ["."] 
    # Get pairs of characters in a sliding window and add to dict
    for ch1, ch2 in zip(chs, chs[1:]):
        ix1 = s_to_i[ch1]
        ix2 = s_to_i[ch2]
        N[ix1, ix2] += 1

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.imshow(N, cmap='Blues')

for i in range(27):
    for j in range(27):
        chstr = i_to_s[i] + i_to_s[j]
        plt.text(j, i, chstr, ha="center", va="bottom", color="gray", fontsize='4')
        plt.text(j, i, N[i, j].item(), ha="center", va="top", color="gray", fontsize='4')

plt.axis('off')

In [ ]:
# We do +1 to smooth the model and prevent -inf prob from log
P = (N+1).float()
# Sum each row of the vector, and return sum in column (1) format
# Keep dim is True to hold dimensions and will be used for broadcasting
P /= P.sum(1, keepdim=True) # /= is faster because in place and doesn't create new memory
P[0].sum()

In [ ]:
# Generator for multimonial
g = torch.Generator().manual_seed(214748)

for i in range(10):
    out = []
    ix = 0
    while True:

        p = P[ix] # Made code below more efficient on top
        # p = N[ix].float()
        # Get prob for each p (first letter)
        # p = p / p.sum()
        # print(p)
        
        # draw random samples from a dataset where some items are more likely to be picked than others
        ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
        out.append(i_to_s[ix])
        if ix == 0:
            break
    print(''.join(out))
    

In [ ]:
# GOAL: maximise the likelihood of data w.r.t. model parameters
# equivalent to maximising log likelihood
# equivalent to minimising negative log likelihood
# equivalent to minmising the average nll 

# We use log because log(a * b * c) = log(a) + log(b) + log(c)

log_likelihood = 0.0
n = 0

for w in words:
    # Add start and ending characters
    chs = ["."] + list(w) + ["."] 
    # Get pairs of characters in a sliding window and add to dict
    for ch1, ch2 in zip(chs, chs[1:]):
        ix1 = s_to_i[ch1]
        ix2 = s_to_i[ch2]
        prob = P[ix1, ix2]
        logprob = torch.log(prob)
        log_likelihood += logprob
        n += 1
        print(f'{ch1}{ch2}: {prob:.4f}{logprob:.4f}')

print(f'{log_likelihood=}')
nll = -log_likelihood
print(f'{nll=}')
print(f'{nll/n=}')


## Full Run

In [ ]:
# create a training set for the bigrams (x,y)
xs, ys = [], []

for w in words:
    # Add start and ending characters
    chs = ["."] + list(w) + ["."] 
    # Get pairs of characters in a sliding window and add to list
    for ch1, ch2 in zip(chs, chs[1:]):
        ix1 = s_to_i[ch1]
        ix2 = s_to_i[ch2]
        print(ch1, ch2)
        xs.append(ix1)
        ys.append(ix2)

xs = torch.tensor(xs)
ys = torch.tensor(ys)
num = xs.nelement()

In [ ]:
import torch.nn.functional as F

# initialise the network - randn is a standard normal distribution with mean 0 and variance 1
W = torch.randn(27, 27, requires_grad=True)

for k in range(100):
    # -- forward pass
    # creates a vector for each letter, index of char = 1, rest = 0
    xenc = F.one_hot(xs, num_classes=27).float()
    logits = (xenc @ W) # predicts log counts
    # this creates the softmax function
    counts = logits.exp()
    probs = counts / counts.sum(1, keepdim=True) # probs for next character
    loss = -probs[torch.arange(num), ys].log().mean() + 0.1 * (W**2).mean() # applying regularisation - way to penalise using large weights

    # -- backward pass
    W.grad = None # set to zero
    loss.backward()

    # -- update
    W.data += -50 * W.grad 
    print(loss)

